# **Retrieval Augmented Generation**

_Taalmodellen zijn heel krachtig, maar zijn qua kennis gelimiteerd tot hun trainingdata en kunnen halucineren. De antwoorden zijn niet gestaafd door een bronvermelding. Vragen over onze eigen documenten kan deze ook niet zomaar beantwoorden. Dit is waar Retrieval Augmented Generation (RAG) een oplossing biedt. Het is een techniek waarbij we onze eigen documenten kunnen bevragen met natuurlijke taal._

<img src="../.github/rag.png" alt="RAG" width="600"/>

---

## **Voorbereiding**

In [ ]:
%pip install -qU langchain_milvus langchain-ollama ollama langchain_community langchain_text_splitters langchain_core pypdf

---

## **Embedder**

<img src="../.github/tekst-embedding.png" alt="Tekst embedding" width="500"/><br>

In [30]:
from langchain_ollama import OllamaEmbeddings

embedder = OllamaEmbeddings(model="llama3")

---

## **Vector database**

Zoals verteld tijdens de theorie, hebben we een **vector database** nodig om onze documenten in op te slaan.

| Opslagmethode     | Gebruik                   |
|-------------------|---------------------------|
| ☁️ Cloud             | Productie                 |
| 🐋 Docker container  | Productie (eigen server)  |
| 📁 Lokaal bestand    | Development               |
| 🕓 In-memory         | Development               |

Wij gaan **Milvus** (open-source) gebruiken als lokaal bestand.<br>

### **1. Aanmaken van database**

Zoals verteld in de theorie, moet je **documenten eerst embedden** alvorens ze toe te voegen aan de database.

<img src="../.github/tekst-vectors-database.png" alt="Tekst embedding" width="500"/><br>

LangChain staat ons toe om de **embedder al mee te geven** bij het aanmaken van de database client. <br>
We kunnen hierdoor later **rechtstreeks documenten toevoegen** zonder ze eerst manueel te embedden, LangChain zal dit automatisch doen!

In [ ]:
from langchain_milvus import Milvus

URI = "../milvus.db"
vectordb = Milvus(embedding_function=embedder, connection_args={"uri": URI})
#                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^

### **2. Uitlezen van PDF**

<img src="../.github/tekst-extractie.png" alt="Tekst extractie" width="400"/>

LangChain heeft een heleboel **voorgemaakte "data loaders"** om tekst uit bestanden te kunnen halen. <br>
Zo is er eentje die gespecialiseerd is in het uitlezen van PDF bestanden.

In [32]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Blogpost.pdf")
documents = loader.load()

### **3. Opsplitsen in kleinere sub-documenten**
We kunnen in theorie de hele PDF omzetten naar 1 vector, maar daar zou heel **weinig informatie** in vervat zitten. <br>
Om gerichter te kunnen zoeken, worden de documenten **opgesplitst in kleinere stukken** of "chunks". <br>
LangChain beschikt opnieuw over **voorgemaakte "text splitters"** die dit voor ons kunnen doen.

In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configureer een text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)

# Splits de documenten in chunks
chunks = splitter.split_documents(documents)

### **4. Documenten in database stoppen**

<img src="../.github/documenten-in-database-stoppen.png" alt="Documenten in database stoppen" width="500"/><br>

*Omdat we de embedder al hebben meegegeven bij het aanmaken van de database client, kunnen we rechtstreeks de documenten toevoegen.* <br>
*LangChain zal ze automatisch embedden voor ons.*

In [ ]:
vectordb.add_documents(chunks)

# Alternatieve manier met manuele embedding tussenstap:
#
# texts = [chunk.page_content for chunk in chunks]
# vectors = embedder.embed_documents(chunks)
# vectordb.add_embeddings(texts, vectors)

---

## **Database bevragen** *(= Retrieval)*

### **5.1 Database bevragen**

<img src="../.github/relevante-docs-zoeken.png" alt="Documenten in database stoppen" width="500"/><br>

*Omdat we de embedder al hebben meegegeven bij het aanmaken van de database client, kunnen we rechtstreeks de vraag in tekst-vorm meegeven.* <br>
*LangChain zal deze automatisch embedden alvorens de zoekopdracht uit te voeren.*

In [19]:
question = "Waarover gaat de blogpost?"

# Zoeken in de database naar gerelateerde documenten
documents_found = vectordb.search(question, search_type="similarity", k=3)


# Alternatieve manier met manuele embedding tussenstap:
#
# vector = embedder.embed_query(question)
# documents = vectordb.similarity_search_by_vector(vector, k=3)

### **5.2 Database bevragen** *(alternatieve manier)*

Een **retriever** is een soort **LangChain component** die conceptueel ruimer is dan enkel zoeken in vector databases. <br>
Ze kunnen ook andere bronnen aanspreken zoals het internet (als je dat zou willen). <br>
Dit is meer best-practice in LangChain. <br>

In [ ]:
question = "Waarover gaat de blogpost?"

# Maak een LangChain retriever object
retriever = vectordb.as_retriever()

# Bevraag de database met de retriever
documents_found = retriever.invoke(question)

---

## **Antwoord forumuleren** *(= Generation)*

### **6. Vraag & documenten aan taalmodel geven**

<img src="../.github/antwoord-formuleren.png" alt="Tekst extractie" width="600"/>

LangChain standaardiseerd het proces om taalmodellen in te laden en aan te roepen:

In [15]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3")

Er zijn ook **voorgemaakte prompts** beschikbaar die je kan importeren en **geoptimaliseerd zijn voor RAG**. <br>
Wij gaan zelf eentje opstellen, voel je vrij om deze aan te passen om bv. een andere schrijfstijl te gebruiken.

In [37]:
from langchain_core.prompts import PromptTemplate

rag_template = """
Je bent een assistent die vragen beantwoordt.
Gebruik de volgende stukjes context om de vraag te beantwoorden.
Vermeld altijd de bestandsnamen als bron bij je antwoorden.
Als je het antwoord niet letterlijk in de context staat, zeg dan dat je het niet weet.
Gebruik maximaal drie zinnen en houd het antwoord beknopt.

Vraag: {question}

Context:\n{context}

Antwoord: """

prompt_template = PromptTemplate.from_template(rag_template)

In [ ]:
# We voegen de gevonden documenten samen tot 1 tekst en vermelden de bijhorende bestandsnamen
context = ""
for document in documents_found:
    filename = document.metadata["source"]
    page = document.metadata["page_label"]
    text = document.page_content.strip()
    context += f"[{filename} - pagina {page}]\n{text}\n\n"

# We injecteren de vraag en context in het prompt sjabloon
prompt = prompt_template.invoke({"question": question, "context": context})

# We geven de prompt aan de LLM
answer = llm.invoke(prompt)

print(answer.content)

---

### **❓ Vraag:** Zou je een OpenAI embedder kunnen combineren met een LLM dat niet van OpenAI is (in een RAG context)?

<details>
  <summary>Antwoord onthullen (klikken)</summary>
  
  **✅ Ja, dat kan!** <br>

  Het embedden wordt enkel en alleen maar toegepast om **semantisch te kunnen zoeken** in de database naar documenten die betrekking hebben tot de vraag. <br>
  We bewaren in de database **niet alleen de vector**, maar ook de bijhorende **oorsponkelijke chunk/tekst**. <br>
  Het is die **zuivere tekst** die we gebruiken in een **prompt voor het taalmodel**. <br>
  Het gebruikte embedding model staat dus volledig los van het taalmodel. <br>

  Je kan perfect mixen en matchen met verschillende databases, taalmodellen en embedders. <br>

</details>